# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [14]:
# Imports here

import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms,utils, models
from torch.utils.data import Dataset, DataLoader
import helper

import matplotlib.pyplot as plt
from collections import OrderedDict

import json


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from workspace_utils import active_session

# with active_session():
#     # do long-running work here

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [15]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [16]:
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms_train = transforms.Compose([transforms.RandomRotation([-30,30]),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

data_transforms_test_eval = transforms.Compose([transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])    
                                                
# TODO: Load the datasets with ImageFolder

trainset = datasets.ImageFolder(train_dir, transform=data_transforms_train)
testset = datasets.ImageFolder(test_dir, transform=data_transforms_test_eval)
evalset = datasets.ImageFolder(valid_dir, transform=data_transforms_test_eval)   
                                                
                                                
# TODO: Using the image datasets and the trainforms, define the dataloaders

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,shuffle=True)                                                
evalloader = torch.utils.data.DataLoader(evalset, batch_size=64,shuffle=True)                                                

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [17]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
class_to_label=trainset.class_to_idx   
label_to_class = {v: k for k, v in class_to_label.items()}



In [18]:
# Run this to test your data loader
images, labels = next(iter(trainloader))
#helper.imshow(images[0], normalize=False)
print(labels[0].item())
print(images[0].size())
# string_=str(labels[0].item())
# print(str(labels[0].item()))

# class_=str(label_to_class[labels[0].item()])
# print(class_)
# print(cat_to_name[class_])


def label_to_name_function(label_,cat_to_name,label_to_class):
    
    class_=str(label_to_class[label_.item()])
    name_=cat_to_name[class_]
    return name_
    
print(label_to_name_function(labels[0],cat_to_name,label_to_class))

label_to_name = {k: cat_to_name[v] for k, v in label_to_class.items()}
#print(label_to_name)

54
torch.Size([3, 224, 224])
bishop of llandaff


# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [19]:
# TODO: Build and train your network

model = models.resnet152(pretrained=True)


for param in model.parameters():
    param.requires_grad = False
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [20]:
# TODO: Build and train your network

model = models.resnet152(pretrained=True)
input_size = 2048
hidden_sizes = [1024,512,256]
output_size = 102

for param in model.parameters():
    param.requires_grad = False


classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(input_size, hidden_sizes[0])),
                          ('relu1', nn.ReLU()),
                          ('dropout1',nn.Dropout(p=0.2)),
                          ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
                          ('relu2', nn.ReLU()),
                          ('dropout2',nn.Dropout(p=0.2)),
                          ('fc3', nn.Linear(hidden_sizes[1], hidden_sizes[2])),
                          ('relu3', nn.ReLU()),
                          ('dropout3',nn.Dropout(p=0.1)),
                          ('fc4', nn.Linear(hidden_sizes[2], output_size)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
        
model.fc = classifier   #last layer of resnet called fc not classifier so removing model.classifier with model.fc

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3) #, betas=(0.9, 0.999), eps=1e-8,

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [22]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def training(model, trainloader, evalloader, criterion, optimizer,device,epochs, print_batch):
    
    model=model
    trainloader=trainloader
    evalloader=evalloader
    criterion=criterion
    optimizer=optimizer
    device=device
    
    model.to(device)

    
    
    for e in range(epochs):
        
        training_loss_running = 0.0
        accuracy_running=0.0
        count__=0
        
        model.train()   #with dropouts
        for step, (inputs, labels) in enumerate(trainloader):
                        
            inputs=inputs.to(device)
            labels=labels.to(device)
                        
            optimizer.zero_grad()
            
            output = model.forward(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            out_exp = torch.exp(output)
            
            count_vec = (labels.data == out_exp.max(1)[1])  #true count vector
            count__ +=labels.size(0)
            
            accuracy_running += count_vec.type_as(torch.FloatTensor()).sum().item()
            
            training_loss_running += loss.item()

            if step % print_batch == 0:
                
                              
                #gradient turned off and model_eval inside of validation function
                test_loss, test_accuracy = validation(model, evalloader, criterion,device)  
                
#                 print("epoch %d / %d, accuracy_run_train: %f, loss_run_train: %f, eval_accuracy: %f, eval_loss: %f"
#                       % (e+1,epochs,accuracy_running/print_batch,training_loss_running/print_batch,
#                         test_accuracy,test_loss))

                print("epoch %d / %d, accuracy_run_train: %f, loss_run_train: %f, eval_accuracy: %f, eval_loss: %f"
                      % (e+1,epochs,accuracy_running*100/count__,training_loss_running/count__,
                        test_accuracy,test_loss))                
                
                accuracy_running=0.0
                training_loss_running=0.0
                count__=0.0
                # turn on drop out grad for training
                model.train()

In [23]:
def validation(model, loader_, criterion,device):
    
    model=model
    loader_=loader_
    criterion=criterion
    device=device
    count_=0
    model.eval()   #evaluation mode to only do forward without dropout
    
    with torch.no_grad():
        
        accuracy = 0
        test_loss = 0
        for step, (inputs, labels) in enumerate(loader_):
            
            inputs=inputs.to(device)
            labels=labels.to(device)
            
            output = model.forward(inputs)
            test_loss += criterion(output, labels).item()

            out_exp = torch.exp(output)    #for nn.NLLLoss()
            
            count_vec = (labels.data == out_exp.max(1)[1])
            count_ +=labels.size(0)
            # Accuracy is number of correct predictions divided by all predictions, just take the mean (no is not, you need to divide by sample size here to be called accuracy)
            accuracy += count_vec.type_as(torch.FloatTensor()).sum().item()  #count of true
            
        test_loss=test_loss/(count_)
        accuracy=accuracy*100.0/(count_)
        
    return test_loss, accuracy

In [24]:
with active_session():
    
     # do long-running work here
    training(model, trainloader, evalloader, criterion, optimizer,device,10, 40)

epoch 1 / 10, accuracy_run_train: 0.000000, loss_run_train: 0.072126, eval_accuracy: 1.100244, eval_loss: 0.073336
epoch 1 / 10, accuracy_run_train: 3.710938, loss_run_train: 0.070413, eval_accuracy: 5.867971, eval_loss: 0.068216
epoch 1 / 10, accuracy_run_train: 11.250000, loss_run_train: 0.062448, eval_accuracy: 22.860636, eval_loss: 0.051883
epoch 2 / 10, accuracy_run_train: 34.375000, loss_run_train: 0.039545, eval_accuracy: 36.063570, eval_loss: 0.041329
epoch 2 / 10, accuracy_run_train: 32.343750, loss_run_train: 0.041352, eval_accuracy: 44.376528, eval_loss: 0.031407
epoch 2 / 10, accuracy_run_train: 42.226562, loss_run_train: 0.033628, eval_accuracy: 57.212714, eval_loss: 0.024075
epoch 3 / 10, accuracy_run_train: 43.750000, loss_run_train: 0.031861, eval_accuracy: 52.200489, eval_loss: 0.027377
epoch 3 / 10, accuracy_run_train: 49.960938, loss_run_train: 0.028176, eval_accuracy: 64.792176, eval_loss: 0.018821
epoch 3 / 10, accuracy_run_train: 55.195312, loss_run_train: 0.02522

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [26]:
# TODO: Do validation on the test set
test_loss, test_accuracy=validation(model, testloader, criterion,device)
print("test_loss: %f , test_accuracy : %f " % (test_loss, test_accuracy))

test_loss: 0.006856 , test_accuracy : 87.423687 


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [27]:
# TODO: Save the checkpoint 

checkpoint = {'input_size': 2048,
              'output_size': 102,
              'hidden_layers': [1024,512,256],
              'model_name':'resnet152',
              'optimizer_name':'Adam',
              'model_base':model,
              'model_classifier':model.fc,
              'optimizer':optimizer,
              'model_state_dict': model.state_dict(),'optimizer_state_dict':optimizer.state_dict,
              'optimizer_state_dict_':optimizer.state_dict(),'criterion_loss_function':criterion,
              'epochs':10,'printing_batch':40,'device':'cuda',
              'class_to_label': trainset.class_to_idx, 'label_to_name':label_to_name}   

# "state_dic" keep the weights of the model only for a specific structure
# 'class_to_label' keep conversion to 
# 'label_to_name' use json file to get the name of label (by using class_to_idx)
# 'optimizer_state_dict' general shape of optimizer
# 'optimizer_state_dict()' content of optimizer
# 'epochs' number of epochs run
# 'printing_batch' how often print
torch.save(checkpoint, 'checkpoint_adam_resnet152_10epoch_opti_name.pth')

input_size = 2048
hidden_sizes = [1024,512,256]
output_size = 102

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [11]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

#model = models.resnet152(pretrained=True)

def load_checkpoint(filepath):
    
    checkpoint = torch.load(filepath)
    model=checkpoint['model_base']
    #for param in model.parameters():
    #    param.requires_grad = False

    model.fc = checkpoint['model_classifier']
    model.load_state_dict(checkpoint['model_state_dict'])             #keeping weight and layers
    model.class_to_idx=checkpoint['class_to_label']
    model=model.cuda()
    model=model.eval()
    return model,checkpoint
    



In [12]:
model_,checkpoint_ = load_checkpoint('checkpoint_1strun_adam_resnet152_10epoch.pth')
#criterion_=checkpoint_['criterion_loss_function']
device_=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion_=nn.NLLLoss()
test_loss, test_accuracy=validation(model_, testloader, criterion_,device_)
print(test_loss, test_accuracy) #0.01810573891114548 0.6715506715506715

0.008514491411355825 84.004884004884


In [13]:
print(model_.fc.parameters())

<generator object Module.parameters at 0x7f535ea87f10>


In [14]:
#model_,checkpoint_ = load_checkpoint('checkpoint_1strun_adam_resnet152_10epoch_opti.pth')
model_,checkpoint_ = load_checkpoint('checkpoint_1strun_adam_resnet152_10epoch.pth')
optimizer_=optim.Adam(model_.fc.parameters(), lr=1e-3)
#model_.train()
#optimizer_=checkpoint_['optimizer']                  #optim.Adam(model_.parameters(), lr=1e-3)    #checkpoint_['optimizer']
#optimizer_.state_dict=checkpoint_['optimizer_state_dict']
#optimizer_=checkpoint_['optimizer']
#optimizer_.load_state_dict(checkpoint['optimizer_state_dict_'])
device_=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion_=nn.NLLLoss()
training(model_, trainloader, evalloader, criterion_, optimizer_,device_,4, 40)

epoch 1 / 4, accuracy_run_train: 70.312500, loss_run_train: 0.018052, eval_accuracy: 85.330073, eval_loss: 0.008043
epoch 1 / 4, accuracy_run_train: 75.976562, loss_run_train: 0.012474, eval_accuracy: 88.386308, eval_loss: 0.007073
epoch 1 / 4, accuracy_run_train: 75.898438, loss_run_train: 0.013112, eval_accuracy: 86.430318, eval_loss: 0.008072
epoch 2 / 4, accuracy_run_train: 79.687500, loss_run_train: 0.014435, eval_accuracy: 87.652812, eval_loss: 0.007234
epoch 2 / 4, accuracy_run_train: 79.335938, loss_run_train: 0.011739, eval_accuracy: 87.897311, eval_loss: 0.006881
epoch 2 / 4, accuracy_run_train: 76.367188, loss_run_train: 0.012624, eval_accuracy: 87.286064, eval_loss: 0.008015
epoch 3 / 4, accuracy_run_train: 79.687500, loss_run_train: 0.011098, eval_accuracy: 88.630807, eval_loss: 0.007047
epoch 3 / 4, accuracy_run_train: 77.851562, loss_run_train: 0.011628, eval_accuracy: 88.630807, eval_loss: 0.006880
epoch 3 / 4, accuracy_run_train: 79.921875, loss_run_train: 0.011000, ev

In [15]:
test_loss, test_accuracy=validation(model_, testloader, criterion_,device_)
print(test_loss, test_accuracy) #0.01810573891114548 0.6715506715506715

0.00639405249821543 88.64468864468864


In [17]:
#model_,checkpoint_ = load_checkpoint('checkpoint_1strun_adam_resnet152_10epoch_opti.pth')
model_,checkpoint_ = load_checkpoint('checkpoint_1strun_adam_resnet152_10epoch_opti.pth')
optimizer_=checkpoint_['optimizer']    #optim.Adam(model_.fc.parameters(), lr=1e-3)
#model_.train()
#optimizer_=checkpoint_['optimizer']                  #optim.Adam(model_.parameters(), lr=1e-3)    #checkpoint_['optimizer']
#optimizer_.state_dict=checkpoint_['optimizer_state_dict']
#optimizer_=checkpoint_['optimizer']
#optimizer_.load_state_dict(checkpoint['optimizer_state_dict_'])
device_=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion_=nn.NLLLoss()
training(model_, trainloader, evalloader, criterion_, optimizer_,device_,4, 40)

epoch 1 / 4, accuracy_run_train: 79.687500, loss_run_train: 0.010478, eval_accuracy: 85.207824, eval_loss: 0.008279
epoch 1 / 4, accuracy_run_train: 77.578125, loss_run_train: 0.012081, eval_accuracy: 87.041565, eval_loss: 0.007135
epoch 1 / 4, accuracy_run_train: 76.992188, loss_run_train: 0.012636, eval_accuracy: 86.552567, eval_loss: 0.007401
epoch 2 / 4, accuracy_run_train: 82.812500, loss_run_train: 0.010574, eval_accuracy: 87.897311, eval_loss: 0.007265
epoch 2 / 4, accuracy_run_train: 78.242188, loss_run_train: 0.012068, eval_accuracy: 88.019560, eval_loss: 0.006619
epoch 2 / 4, accuracy_run_train: 79.453125, loss_run_train: 0.011511, eval_accuracy: 87.775061, eval_loss: 0.007042
epoch 3 / 4, accuracy_run_train: 79.687500, loss_run_train: 0.011016, eval_accuracy: 88.141809, eval_loss: 0.007671
epoch 3 / 4, accuracy_run_train: 79.531250, loss_run_train: 0.011217, eval_accuracy: 88.386308, eval_loss: 0.006776
epoch 3 / 4, accuracy_run_train: 80.273438, loss_run_train: 0.010858, ev

In [18]:
#model_,checkpoint_ = load_checkpoint('checkpoint_1strun_adam_resnet152_10epoch_opti.pth')
model_,checkpoint_ = load_checkpoint('checkpoint_1strun_adam_resnet152_10epoch_opti.pth')
optimizer_=checkpoint_['optimizer']    #optim.Adam(model_.fc.parameters(), lr=1e-3)
#model_.train()
#optimizer_=checkpoint_['optimizer']                  #optim.Adam(model_.parameters(), lr=1e-3)    #checkpoint_['optimizer']
#optimizer_.state_dict=checkpoint_['optimizer_state_dict']
#optimizer_=checkpoint_['optimizer']
#optimizer_.load_state_dict(checkpoint['optimizer_state_dict_'])
device_=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion_= checkpoint_['criterion_loss_function']                   #           nn.NLLLoss()
training(model_, trainloader, evalloader, criterion_, optimizer_,device_,4, 40)

epoch 1 / 4, accuracy_run_train: 71.875000, loss_run_train: 0.013762, eval_accuracy: 85.085575, eval_loss: 0.008196
epoch 1 / 4, accuracy_run_train: 76.562500, loss_run_train: 0.012864, eval_accuracy: 86.797066, eval_loss: 0.006869
epoch 1 / 4, accuracy_run_train: 76.132812, loss_run_train: 0.012905, eval_accuracy: 88.386308, eval_loss: 0.006842
epoch 2 / 4, accuracy_run_train: 78.125000, loss_run_train: 0.010731, eval_accuracy: 88.141809, eval_loss: 0.006550
epoch 2 / 4, accuracy_run_train: 78.632812, loss_run_train: 0.011768, eval_accuracy: 88.630807, eval_loss: 0.006951
epoch 2 / 4, accuracy_run_train: 79.335938, loss_run_train: 0.011318, eval_accuracy: 89.486553, eval_loss: 0.006778
epoch 3 / 4, accuracy_run_train: 79.687500, loss_run_train: 0.010576, eval_accuracy: 86.797066, eval_loss: 0.007426
epoch 3 / 4, accuracy_run_train: 77.812500, loss_run_train: 0.012102, eval_accuracy: 89.119804, eval_loss: 0.006342
epoch 3 / 4, accuracy_run_train: 78.125000, loss_run_train: 0.011468, ev

In [19]:
test_loss, test_accuracy=validation(model_, testloader, criterion_,device_)
print(test_loss, test_accuracy) #0.01810573891114548 0.6715506715506715

0.0073578101702225516 87.3015873015873


In [39]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

#model = models.resnet152(pretrained=True)

#this version doesn't need optimizer to get loaded just rebuild it outside again
def load_checkpoint_new(filepath):   #this also work   may be add the name of model to dictionray and run for longer time?
    
       
    checkpoint = torch.load(filepath)
    
    model = models.resnet152(pretrained=True)


    for param in model.parameters():
        
        param.requires_grad = False
    
    model.class_to_idx=checkpoint['class_to_label']
    model.fc = checkpoint['model_classifier']

    model.load_state_dict(checkpoint['model_state_dict'])             #keeping weight and layers
    
    model=model.cuda()
    model=model.eval()
    return model,checkpoint
    



In [31]:
#With this rebuild the model by knowing it is resnet152
model_,checkpoint_ = load_checkpoint_new('checkpoint_adam_resnet152_10epoch_opti_name.pth')
optimizer_=optim.Adam(model_.fc.parameters(), lr=1e-3)
device_=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion_= checkpoint_['criterion_loss_function']                   #           nn.NLLLoss()

test_loss, test_accuracy=validation(model_, testloader, criterion_,device_)
print(test_loss, test_accuracy)

training(model_, trainloader, evalloader, criterion_, optimizer_,device_,1, 40)

0.006820002427467933 87.42368742368743
epoch 1 / 1, accuracy_run_train: 79.687500, loss_run_train: 0.013609, eval_accuracy: 88.875306, eval_loss: 0.006754
epoch 1 / 1, accuracy_run_train: 76.562500, loss_run_train: 0.012799, eval_accuracy: 86.552567, eval_loss: 0.007455
epoch 1 / 1, accuracy_run_train: 78.828125, loss_run_train: 0.011607, eval_accuracy: 88.997555, eval_loss: 0.006576


In [38]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

#This loads the whole model

def load_checkpoint(filepath):
    
    checkpoint = torch.load(filepath)
    model=checkpoint['model_base']
    #for param in model.parameters():
    #    param.requires_grad = False   #have to turn this of if I want to load optimizer as a whole in the next step

    model.fc = checkpoint['model_classifier']
    model.load_state_dict(checkpoint['model_state_dict'])             #keeping weight and layers
    model.class_to_idx=checkpoint['class_to_label']
    model=model.cuda()
    model=model.eval()
    return model,checkpoint
    



In [34]:
#With this This loads the whole model of resnet152 from checkpoint
model_,checkpoint_ = load_checkpoint('checkpoint_adam_resnet152_10epoch_opti_name.pth')
optimizer_=checkpoint_['optimizer']    #optim.Adam(model_.fc.parameters(), lr=1e-3)

device_=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion_= checkpoint_['criterion_loss_function']                   #           nn.NLLLoss()

test_loss, test_accuracy=validation(model_, testloader, criterion_,device_)
print(test_loss, test_accuracy)

training(model_, trainloader, evalloader, criterion_, optimizer_,device_,1, 40)

0.00680608834539141 87.42368742368743
epoch 1 / 1, accuracy_run_train: 90.625000, loss_run_train: 0.008175, eval_accuracy: 88.753056, eval_loss: 0.006433
epoch 1 / 1, accuracy_run_train: 76.679688, loss_run_train: 0.012290, eval_accuracy: 89.608802, eval_loss: 0.006350
epoch 1 / 1, accuracy_run_train: 79.062500, loss_run_train: 0.011458, eval_accuracy: 89.486553, eval_loss: 0.006468


In [44]:
model_,checkpoint_ = load_checkpoint('checkpoint_adam_resnet152_10epoch_opti_name.pth')
device_=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

image, label = next(iter(testloader))

print(label[0])
#print(image[0])
inputs=image[0]
labels=label[0]

inputs=inputs.to(device_)
labels=labels.to(device_)
print(inputs.size())
#print(labels.size())

model_.eval()
with torch.no_grad():
    img=inputs
    img.unsqueeze_(0)  #   with single image batch size missing, so need to add another dimension  
                       #   https://discuss.pytorch.org/t/expected-stride-to-be-a-single-integer-value-or-a-list/17612/4
    output = model_.forward(img)
    out_exp = torch.exp(output)
    pred=out_exp.max(1)[1]
    pred_list=out_exp
    
print('pred',pred)
print(out_exp.max(1))
print(pred_list)

pred_numpy=pred_list.cpu().numpy()   #need to make to cpu 
#print(pred_numpy)
sort_list=np.argsort(pred_numpy[0])
sort_list_rev=sort_list[::-1]

top_5_list=sort_list[-5:]
top_5_list_rev=sort_list_rev[0:5]
print(top_5_list)
print(top_5_list_rev)
#print(checkpoint_['label_to_name'])
dict_= checkpoint_['label_to_name']
step=0
print(' ')
print('REAL-----',dict_[label[0].item()]) 
for i in top_5_list_rev:
    step +=1
    print('Rank',step,dict_[i])
   

tensor(73)
torch.Size([3, 224, 224])
pred tensor([ 73], device='cuda:0')
(tensor([ 0.9943], device='cuda:0'), tensor([ 73], device='cuda:0'))
tensor([[ 8.5355e-12,  3.8788e-14,  3.8799e-09,  5.0427e-12,  6.1026e-09,
          8.8917e-11,  2.5038e-10,  5.1820e-07,  3.1173e-14,  4.2078e-12,
          2.7255e-11,  3.7945e-10,  5.0184e-09,  4.7921e-12,  2.6022e-10,
          1.0647e-08,  3.9552e-11,  6.2794e-12,  1.4103e-09,  4.0204e-07,
          7.9309e-10,  7.8945e-10,  3.7957e-14,  4.1843e-14,  2.0478e-08,
          7.3904e-11,  5.3879e-12,  3.3003e-09,  6.9957e-11,  5.6613e-13,
          3.7450e-09,  1.1012e-13,  3.8784e-08,  2.1849e-13,  5.7490e-10,
          2.6157e-05,  5.4393e-09,  4.2302e-08,  2.6785e-07,  1.6942e-08,
          9.3041e-10,  2.7368e-10,  8.5610e-08,  2.3219e-09,  4.5502e-11,
          1.2192e-09,  3.1883e-07,  6.2802e-08,  1.4012e-11,  4.6836e-12,
          8.0769e-11,  8.7752e-10,  9.1217e-10,  6.5193e-14,  1.5618e-08,
          3.0786e-11,  2.4936e-14,  6.6975e-

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes